In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive/document_classifier/")

In [ ]:
# !pip install pymupdf

In [ ]:
# %%time
# import fitz
# filepath = "./a2937214-5676-4354-a85c-8c26580b46ab.pdf"

# text = ''
# with fitz.open(filepath ) as doc:
#     for page in doc:
#         text+= page.get_text("text")
# print(text)

In [ ]:
import pickle
import os, re
# from pdf2image import convert_from_path
import string
import nltk
import fitz
nltk.download('stopwords')
# from multiprocessing import Pool

def load_model(filename):
  # load the model from disk
  loaded_model = pickle.load(open(filename, 'rb'))
  return loaded_model

loaded_model = load_model(filename='./finalized_model.pkl')

punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("'")
def remove_punctuations(text):
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    tokens = text.split()
    output= " ".join([i for i in tokens if i not in stopwords])
    return output

# def get_text_from_page(page_number, page):
#   # page = read_pdf.getPage(page_number)
#   # page_content = page.extractText()
#   page_content = page.get_text("text")
#   print(page_content)

#   #Text cleaning
#   page_content = remove_punctuations(page_content)
#   page_content = re.sub(' +', ' ', page_content)
#   page_content = remove_stopwords(page_content)
#   page_content = page_content.lower()

#   return (page_number+1, page_content)


def get_classification(pdf_path):
    #Extract text from each page
    page_info = []
    result = {}
    target_name = ['not_important', 'important']
    with fitz.open(pdf_path) as doc:
      for i, page in enumerate(doc):
          page_info.append((i+1, page.get_text("text")))
    txt_list = [str(txt) for page, txt in page_info]

    # Text cleaning
    txt_process = list(map(remove_punctuations, txt_list))
    txt_process = list(map(lambda x: str(x).replace("  ", " "), txt_process))
    txt_process = list(map(remove_stopwords, txt_process))
    final_txt_list = list(map(lambda x: x.lower(), txt_process))

    #model prediction
    pred_prob = loaded_model.predict_proba(final_txt_list)
    ix = pred_prob.argmax(1)
    prediction = [(target_name[val], round(float(prob[val])*100, 2), info[0]) for val, prob, info in zip(ix, pred_prob, page_info)]

    result['important'] = [{'page_number': page, 'confidence_score': conf} for label, conf, page in prediction if label == 'important']
    result['not_important'] = [{'page_number': page, 'confidence_score': conf} for label, conf, page in prediction if label == 'not_important']

    return {'result': result}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%%time
result = get_classification("./5ee3f827-6ff8-40b2-b4a3-96d1d25325d4.pdf")

CPU times: user 102 ms, sys: 6.84 ms, total: 109 ms
Wall time: 165 ms


In [ ]:
result

{'result': {'important': [{'page_number': 4, 'confidence_score': 100.0}],
  'not_important': [{'page_number': 1, 'confidence_score': 86.81},
   {'page_number': 2, 'confidence_score': 99.27},
   {'page_number': 3, 'confidence_score': 98.61},
   {'page_number': 5, 'confidence_score': 99.73},
   {'page_number': 6, 'confidence_score': 96.07}]}}

In [ ]:
# !pip install pdf2image
# !sudo apt-get install poppler-utils
# !pip install PyPDF2

In [ ]:
# def get_classification(pdf_path):
#     #Extract text from each page
#     # read_pdf = PyPDF2.PdfFileReader(pdf_path)
#     # number_of_pages = read_pdf.getNumPages()
#     page_info = []
#     result = {}
#     target_name = ['not_important', 'important']
#     result_objs = []
#     doc = fitz.open(pdf_path)
#     #Using multiprocessing for extracting text from multiple pages in a pdf file.
#     print("Total Number of CPU cores :", os.cpu_count())
#     with Pool(processes=os.cpu_count() - 1) as pool:
#         # for i, page_number in enumerate(range(number_of_pages)):   # use xrange in Py2
#         # with fitz.open(pdf_path) as doc:
#           for i, page in enumerate(doc):
#               output = pool.apply_async(get_text_from_page, (i, page,))
#               result_objs.append(output)
#               print(output.get)
#           page_info = [res.get() for res in result_objs]
#           final_txt_list = [str(txt) for page, txt in page_info]
#     doc.close()
#     # Text cleaning
#     # txt_process = list(map(remove_punctuations, txt_list))
#     # txt_process = list(map(lambda x: str(x).replace("  ", " "), txt_process))
#     # txt_process = list(map(remove_stopwords, txt_process))
#     # final_txt_list = list(map(lambda x: x.lower(), txt_process))

#     #model prediction
#     pred_prob = loaded_model.predict_proba(final_txt_list)
#     ix = pred_prob.argmax(1)
#     prediction = [(target_name[val], round(float(prob[val])*100, 2), info[0]) for val, prob, info in zip(ix, pred_prob, page_info)]

#     result['important'] = [{'page_number': page, 'confidence_score': conf} for label, conf, page in prediction if label == 'important']
#     result['not_important'] = [{'page_number': page, 'confidence_score': conf} for label, conf, page in prediction if label == 'not_important']

#     return {'result': result}


In [ ]:
# pdf_path = './5ee3f827-6ff8-40b2-b4a3-96d1d25325d4.pdf'
# import PyPDF2, os, re
# from pdf2image import convert_from_path
# filename = pdf_path.split("/")[-1].split(".")[0]
# if not os.path.exists(f"./media/{filename}"):
#     os.makedirs(f"./media/{filename}")
# images = convert_from_path(pdf_path)
# read_pdf = PyPDF2.PdfFileReader(pdf_path)
# number_of_pages = read_pdf.getNumPages()
# for i, page_number in enumerate(range(number_of_pages)):   # use xrange in Py2
#     page = read_pdf.getPage(page_number)
#     page_content = page.extractText()
#     images[i].save(f"./media/{filename}/"+ filename+ '_page_'+ str(i) +'.jpg', 'JPEG')
#     file1 = open(f"./media/{filename}/"+ filename+ '_page_'+ str(i) +'.txt',"w")#append mode
#     page_content = re.sub(' +', ' ', page_content)
#     file1.write(page_content)
#     file1.close()

In [ ]:
# image_list, txt_list = [], []
# for file in os.listdir(f"./media/{filename}/"):
#   if file.endswith('.jpg'):
#       image_list.append(f"./media/{filename}/" + file)
#       f = open(f"./media/{filename}/" + file.replace(".jpg", '.txt'), 'r')
#       txt =  f.read()
#       txt_list.append(txt)
#       f.close()

In [ ]:
# import string

# punct =[]
# punct += list(string.punctuation)
# punct += '’'
# punct.remove("'")
# def remove_punctuations(text):
#     for punctuation in punct:
#         text = text.replace(punctuation, ' ')
#     return text

In [ ]:
# import nltk
# nltk.download('stopwords')
# #Stop words present in the library
# stopwords = nltk.corpus.stopwords.words('english')
# #defining the function to remove stopwords from tokenized text
# def remove_stopwords(text):
#     tokens = text.split()
#     output= " ".join([i for i in tokens if i not in stopwords])
#     return output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# txt_ = list(map(remove_punctuations, txt_list))
# txt_ = list(map(lambda x: str(x).replace("  ", " "), txt_))
# txt_ = list(map(remove_stopwords, txt_))
# final_txt_list = list(map(lambda x: x.lower(), txt_))

In [ ]:
# final_txt_list[0]

'insured details partner details name partner name maheshbhai patel invictus insurance broking services private ltd address partner code 62 tanki faliyu italva navsari gujarat 396445 db56713 mobile number partner mobile number 9106453742 9513312901 policy no partner email 3001 241154739 00 000 hello maheshbhai patel car covered here private car package policy product code 3001 uin irdan115rp0017v01200102 ref no w136545691 date feb 28 2022 corp sup opi 2014 1777'

In [ ]:
# predicted = loaded_model.predict(final_txt_list)

In [ ]:
# predicted

array([0, 0, 0, 1, 0, 0])

In [ ]:
# target_name = ['not_important', 'important']
# pred_prob = loaded_model.predict_proba(final_txt_list)
# ix = pred_prob.argmax(1)
# prediction = [(target_name[val], round(prob[val], 2)) for val, prob in zip(ix, pred_prob)]
# print(prediction)

[('not_important', 0.87), ('not_important', 0.99), ('not_important', 0.98), ('important', 1.0), ('not_important', 1.0), ('not_important', 0.95)]
